In [1]:
import argparse
import json
import os

import numpy as np
import torch

import trainUtils


/home/tyfei/anaconda3/envs/esm3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = "/data2/tyfei/trainresults/ionChannels/ESMC/tryesmc/" 
with open(os.path.join(path, "config.json"), "r") as f:
    configs = json.load(f)

In [16]:
configs

{'train': {'save': 5,
  'epoch': 40,
  'batch_size': 1,
  'accumulate_grad_batches': 8,
  'seed': 1509},
 'augmentation': {'step_points': [-1, 5000, 10000],
  'maskp': [-1, -1, -1],
  'maskpc': [-1, -1, -1],
  'crop': [-1, 0.2, 0.4],
  'tracks': {'seq_t': 1.0}},
 'dataset': {'type': 'balancedesm3',
  'dataset_train_sample': [[900, 2400, 1500, 600, 150, 300],
   [1000, 500, 500, 500, 3000]],
  'dataset_val_sample': [[90, 240, 150, 60, 15, 30], [100, 50, 50, 50, 300]],
  'seed': 1509,
  'tracks': ['seq_t'],
  'train_test_ratio': [0.85, 0.15],
  'pos': ['/data/tyfei/datasets/ion_channel/Interpro_v2/kingdom/Archaea1_label.pkl_v2_1125.pkl',
   '/data/tyfei/datasets/ion_channel/Interpro_v2/kingdom/Bacteria1_label.pkl_v2_1125.pkl',
   '/data/tyfei/datasets/ion_channel/Interpro_v2/kingdom/Eukaryota1_label.pkl_v2_1125.pkl',
   '/data/tyfei/datasets/ion_channel/Interpro_v2/kingdom/Viruses1_label.pkl_v2_1125.pkl',
   '/data/tyfei/datasets/ion_channel/Interpro_v2/kingdom/None1_label.pkl_v2_1125.pk

In [3]:
pretrain_model = trainUtils.loadPretrainModel(configs)

model = trainUtils.buildModel(configs, pretrain_model, None)

In [4]:
ds = trainUtils.loadDataset(configs)

In [5]:
dl = ds.train_dataloader() 
for i, d in enumerate(dl):
    break 
d

get train loader
called new epoch


[{'seq_t': tensor([[ 0, 20,  5, 10, 18,  7,  5,  4,  7,  4,  4,  6,  4,  4,  8,  4,  8,  9,
            4, 13,  5, 12, 16, 10, 10, 14, 15, 12, 16,  7, 19,  8, 10, 21, 14, 14,
            9, 13,  6, 15, 14, 17, 19,  4, 17, 23, 19,  7,  8,  6, 18, 21, 14, 14,
           16, 12,  9, 12, 13,  4,  4, 15, 17,  6,  9, 15, 12, 15,  8,  9, 16,  8,
           13,  4,  8, 18,  8, 15, 13, 22,  8, 18, 19,  4,  4,  8, 21,  5,  9, 18,
           11, 14, 17,  8, 15, 13, 16, 19,  8, 23, 10,  7, 15, 21,  7, 11,  4,  9,
           16, 14, 16, 12,  7, 15, 22, 13, 10, 13,  4,  2]])},
 tensor([[ 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
          -1, -1, -1]]),
 {'seq_t': tensor([[ 0, 20, 20, 13, 12, 12, 11, 10, 11, 16, 10, 17,  4, 15,  4, 18,  4, 18,
           22, 15, 15,  7, 15, 15,  5,  7, 21, 13,  4,  4, 12, 12, 11,  5, 11, 23,
            7, 12,  7,  6,  6,  6, 11, 19, 20, 19, 11, 11, 19,  4, 16, 13, 21,  6,
           23,  7, 12,  6, 11, 11, 23, 12,  8, 12, 11, 10,  2]])}

In [6]:
d[0]["seq_t"].shape

torch.Size([1, 120])

In [7]:
model.forward(d[0])

tensor([[ 0, 20,  5, 10, 18,  7,  5,  4,  7,  4,  4,  6,  4,  4,  8,  4,  8,  9,
          4, 13,  5, 12, 16, 10, 10, 14, 15, 12, 16,  7, 19,  8, 10, 21, 14, 14,
          9, 13,  6, 15, 14, 17, 19,  4, 17, 23, 19,  7,  8,  6, 18, 21, 14, 14,
         16, 12,  9, 12, 13,  4,  4, 15, 17,  6,  9, 15, 12, 15,  8,  9, 16,  8,
         13,  4,  8, 18,  8, 15, 13, 22,  8, 18, 19,  4,  4,  8, 21,  5,  9, 18,
         11, 14, 17,  8, 15, 13, 16, 19,  8, 23, 10,  7, 15, 21,  7, 11,  4,  9,
         16, 14, 16, 12,  7, 15, 22, 13, 10, 13,  4,  2]]) torch.Size([1, 120])


RuntimeError: mat1 and mat2 must have the same dtype, but got Float and BFloat16

In [10]:
from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, LogitsConfig

In [11]:
protein = ESMProtein(sequence="AAAAA")
client = ESMC.from_pretrained("esmc_300m").to("cuda") # or "cpu"
protein_tensor = client.encode(protein)
logits_output = client.logits(
   protein_tensor, LogitsConfig(sequence=True, return_embeddings=True)
)
print(logits_output.logits, logits_output.embeddings)

ForwardTrackData(sequence=tensor([[[-38.0000, -38.0000, -38.0000,  12.6250,  21.6250,  22.2500,  22.0000,
           21.7500,  21.3750,  21.6250,  21.6250,  21.2500,  20.7500,  21.3750,
           21.8750,  20.8750,  20.7500,  20.2500,  20.3750,  20.1250,  21.7500,
           19.8750,  19.7500,  19.2500,  18.2500,   1.2656,  -1.6719,  -3.9688,
          -20.6250, -38.2500, -38.0000, -38.0000, -38.0000, -38.2500, -38.0000,
          -38.0000, -38.2500, -38.0000, -38.2500, -38.0000, -38.0000, -38.2500,
          -38.0000, -38.0000, -38.0000, -38.2500, -38.2500, -38.0000, -38.2500,
          -38.0000, -38.2500, -38.0000, -38.0000, -38.2500, -38.2500, -38.0000,
          -38.0000, -38.0000, -38.0000, -38.0000, -38.2500, -38.0000, -38.0000,
          -38.0000],
         [-40.2500, -40.0000, -40.2500,   5.3438,  20.1250,  19.8750,  18.7500,
           20.6250,  18.7500,  18.3750,  18.6250,  18.5000,  18.2500,  18.0000,
           18.7500,  17.7500,  17.3750,  17.1250,  17.7500,  16.8750,  22

In [13]:
client(sequence_tokens=d[0]["seq_t"].cuda())

ESMCOutput(sequence_logits=tensor([[[-36.2500, -36.2500, -36.2500,  ..., -36.2500, -36.2500, -36.2500],
         [-36.5000, -36.5000, -36.5000,  ..., -36.5000, -36.5000, -36.5000],
         [-32.7500, -32.7500, -32.7500,  ..., -32.7500, -32.7500, -32.7500],
         ...,
         [-34.7500, -34.5000, -34.5000,  ..., -34.5000, -34.5000, -34.5000],
         [-30.0000, -30.0000, -30.0000,  ..., -30.0000, -30.0000, -29.8750],
         [-31.0000, -31.0000, -31.0000,  ..., -31.0000, -31.0000, -31.0000]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<ViewBackward0>), embeddings=tensor([[[ 0.0026,  0.0060,  0.0051,  ...,  0.0038, -0.0041, -0.0036],
         [-0.0266,  0.0315,  0.0239,  ..., -0.0066, -0.0077, -0.0089],
         [-0.0270,  0.0322, -0.0048,  ..., -0.0430, -0.0479,  0.0398],
         ...,
         [-0.0444,  0.0344,  0.0262,  ...,  0.0153, -0.0640,  0.0262],
         [-0.0535,  0.0649, -0.0220,  ..., -0.0165, -0.0659,  0.0044],
         [-0.0203,  0.0305,  0.0304,  ..., 

In [ ]:
from esm.utils.sampling import _BatchedESMProteinTensor

In [7]:
client.forward(sequence_tokens = protein_tensor.sequence.unsqueeze(0))

ESMCOutput(sequence_logits=tensor([[[-38.0000, -38.0000, -38.0000,  12.6250,  21.6250,  22.2500,  22.0000,
           21.7500,  21.3750,  21.6250,  21.6250,  21.2500,  20.7500,  21.3750,
           21.8750,  20.8750,  20.7500,  20.2500,  20.3750,  20.1250,  21.7500,
           19.8750,  19.7500,  19.2500,  18.2500,   1.2656,  -1.6719,  -3.9688,
          -20.6250, -38.2500, -38.0000, -38.0000, -38.0000, -38.2500, -38.0000,
          -38.0000, -38.2500, -38.0000, -38.2500, -38.0000, -38.0000, -38.2500,
          -38.0000, -38.0000, -38.0000, -38.2500, -38.2500, -38.0000, -38.2500,
          -38.0000, -38.2500, -38.0000, -38.0000, -38.2500, -38.2500, -38.0000,
          -38.0000, -38.0000, -38.0000, -38.0000, -38.2500, -38.0000, -38.0000,
          -38.0000],
         [-40.2500, -40.0000, -40.2500,   5.3438,  20.1250,  19.8750,  18.7500,
           20.6250,  18.7500,  18.3750,  18.6250,  18.5000,  18.2500,  18.0000,
           18.7500,  17.7500,  17.3750,  17.1250,  17.7500,  16.8750,  2